In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from lightning_modules.diffusion_prior import DiffusionPriorTask
import torch
import matplotlib.pyplot as plt

In [7]:
task = DiffusionPriorTask('scog7g95')

Loaded dataset : (11288/1000)
Loaded checkpoing : last.ckpt
Loaded checkpoing : last.ckpt


In [10]:
task.base_task

MetaLearningTask(
  (model): MetaLearner(
    (decoder): TransformerDecoder(
      (token_emb): TokenEmbedding(
        (emb): Embedding(51, 256)
      )
      (pos_emb): ScaledSinusoidalEmbedding()
      (post_emb_norm): Identity()
      (emb_dropout): Dropout(p=0.0, inplace=False)
      (project_emb): Identity()
      (attn_layers): Decoder(
        (layers): ModuleList(
          (0): ModuleList(
            (0): ModuleList(
              (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (1-2): 2 x None
            )
            (1): Attention(
              (to_q): Linear(in_features=256, out_features=512, bias=False)
              (to_k): Linear(in_features=256, out_features=512, bias=False)
              (to_v): Linear(in_features=256, out_features=512, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
              (to_out): Linear(in_features=512, out_features=256, bias=False)
            )
            (2): Residual()
          )
        